<a href="https://colab.research.google.com/github/Dkepffl/2022-2-ESAA/blob/main/Assignment/Assignment221014_text2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **CHAPTER 08 텍스트 분석**
___


In [3]:
# import packages
import pandas as pd

## **04 텍스트 분류 실습-20 뉴스그룹 분류**
___
- 사이킷런이 내부 예제 데이터인 20 뉴스그룹 데이터셋을 이용해 텍스트 분류를 적용해보자.
- 텍스트 분류는 특정 문서의 분류를 학습 데이터를 통해 학습해 모델을 생성한 뒤 이 학습 모델을 이용해 다른 문서의 분류를 예측하는 것이다.
- 텍스트를 기반으로 분류를 수행할 때는 먼저 텍스트를 정규화한 뒤 피처 벡터화를 적용한다. 그리고 적합한 머신러닝 알고리즘을 적용해 분류를 학습/예측/평가합니다.
  - 텍스트를 피처 벡터화로 변환하면 일반적으로 희소 행렬 형태가 되는데, 로지스틱 회귀, 선형 서포트 벡터 머신, 나이브 베이즈 등이 이러한 형태의 데이터를 잘 분류한다. 이 중 로지스틱 회귀를 이용해 분류를 수행할 예정이다.
  - 카운트 기반과 TF-IDF 기반 벡터화를 적용한다.



### | **텍스트 정규화**
- `fetch_20newgroups()`를 이용하여 데이터를 로딩한다.



In [1]:
from sklearn.datasets import fetch_20newsgroups

news_data = fetch_20newsgroups(subset='all',random_state=156)

- `fetch_20newgroups()`는 사이킷런의 다른 데이터셋 예제처럼 파이썬 딕셔너리와 유사한 Bunch 객체를 반환한다. 어떠한 key 값을 가지고 있는지 확인해보자.

In [2]:
print(news_data.keys())

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])


- 다음으로 Target클래스가 어떻게 구성되어 있는지 확인해보자.

In [4]:
print('target 클래스의 값과 분포도 \n',pd.Series(news_data.target).value_counts().sort_index())
print('target 클래스의 이름들 \n',news_data.target_names)

target 클래스의 값과 분포도 
 0     799
1     973
2     985
3     982
4     963
5     988
6     975
7     990
8     996
9     994
10    999
11    991
12    984
13    990
14    987
15    997
16    910
17    940
18    775
19    628
dtype: int64
target 클래스의 이름들 
 ['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


- Target 클래스의 값은 0부터 19까지 20개로 구성되어 있으며, 위의 출력 결과처럼 주어져 있다.
- 개별 데이터가 텍스트로 어떻게 구성되어 있는지 데이터를 한 개만 추출해 값을 확인해보자.

In [5]:
print(news_data.data[0])

From: egreen@east.sun.com (Ed Green - Pixel Cruncher)
Subject: Re: Observation re: helmets
Organization: Sun Microsystems, RTP, NC
Lines: 21
Distribution: world
Reply-To: egreen@east.sun.com
NNTP-Posting-Host: laser.east.sun.com

In article 211353@mavenry.altcit.eskimo.com, maven@mavenry.altcit.eskimo.com (Norman Hamer) writes:
> 
> The question for the day is re: passenger helmets, if you don't know for 
>certain who's gonna ride with you (like say you meet them at a .... church 
>meeting, yeah, that's the ticket)... What are some guidelines? Should I just 
>pick up another shoei in my size to have a backup helmet (XL), or should I 
>maybe get an inexpensive one of a smaller size to accomodate my likely 
>passenger? 

If your primary concern is protecting the passenger in the event of a
crash, have him or her fitted for a helmet that is their size.  If your
primary concern is complying with stupid helmet laws, carry a real big
spare (you can put a big or small head in a big helmet, bu

- 텍스트 데이터를 확인해보면, 뉴스그룹 기사의 내용뿐만 아니라 제목, 작성자, 소속, 이메일 등의 다양한 정보를 가지고 있다.
- 제목과 소속, 이메일 주소 등의 헤더와 푸터 정보들은 Target 클래스 값과 유사한 데이터를 가지고 있는 경우가 많아, 대부분의 ML 알고리즘에서 높은 성능을 보인다. 여기서는 순수한 텍스트만으로 구성된 내용을 제외한 다른 정보는 제거한다.
  + `remove` 파라미터를 이용하여 헤더(header), 푸터(footer) 등을 제거할 수 있다.
  + `fetch_20newsgroups()`는 subset 파라미터를 이용해 학습 데이터셋과 테스트 데이터셋을 분리해 저장할 수 있다.


In [6]:
from sklearn.datasets import fetch_20newsgroups

# subset='train'으로 학습용(Train) 데이터만 추출, remove=('headers','footers','quotes')로 내용만 추출
train_news= fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'), random_state=156)
X_train = train_news.data
y_train = train_news.target

# subset='test'으로 테스트(Test) 데이터만 추출, remove=('headers', 'footers', 'quotes')로 내용만 추출
test_news= fetch_20newsgroups(subset='test',remove=('headers', 'footers','quotes'),random_state=156)
X_test = test_news.data
y_test = test_news.target
print('학습 데이터 크기 {0} , 테스트 데이터 크기 {1}'.format(len(train_news.data) , len(test_news.data)))

학습 데이터 크기 11314 , 테스트 데이터 크기 7532


### | **피처 벡터화 변환과 머신러닝 모델 학습/예측/평가**
- 학습 데이터는 11314개의 뉴스그룹 문서가 리스트 형태로 주어지고, 테스트 데이터는 7532개의 문서가 역시 리스트 형태로 주어졌다.
- `CountVectorizer`를 이용해 학습 데이터의 텍스트를 피처 벡터화하자.
- 테스트 데이터 역시 피처 벡터화를 수행하는데, 테스트 데이터에서 `CountVectorizer`를 적용할 때 반드시 학습 데이터를 이용해 `fit()`이 수행된 객체를 이용하여 테스트 데이터를 변환(transform)해야 한다.
  + 그래야 학습 시 설정된 `CountVectorizer`의 피처 개수와 테스트 데이터의 피처 개수가 같아지기 때문이다.
  + 테스트 데이터의 피처 벡터화는 학습 데이터에 사용된 `CountCextorizer` 객체 변수인 `cnt_vect.transform()`을 이용해 변환한다.
  + 같은 이유로 테스트 데이터의 피처 벡터화 시 `fit_transform()`을 사용하면 안된다.


In [8]:
from sklearn.feature_extraction.text import CountVectorizer

# Count Vectorization으로 feature extraction 변환 수행. 
cnt_vect = CountVectorizer()
cnt_vect.fit(X_train)
X_train_cnt_vect = cnt_vect.transform(X_train)

# 학습 데이터로 fit( )된 CountVectorizer를 이용하여 테스트 데이터를 feature extraction 변환 수행. 
X_test_cnt_vect = cnt_vect.transform(X_test)

print('학습 데이터 Text의 CountVectorizer Shape:',X_train_cnt_vect.shape)

학습 데이터 Text의 CountVectorizer Shape: (11314, 101631)


- 학습 데이터를 `CountVectorizer`로 피처 추출한 결과, 11314개의 문서에서 피처, 즉 단어가 101631개 만들어졌다.
- 이렇게 피처 벡터화된 데이터에 로지스틱 회귀를 적용해 뉴스 그룹에 대한 분류를 예측해보자.

In [9]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# LogisticRegression을 이용하여 학습/예측/평가 수행. 
lr_clf = LogisticRegression()
lr_clf.fit(X_train_cnt_vect , y_train)
pred = lr_clf.predict(X_test_cnt_vect)

print('CountVectorized Logistic Regression 의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test,pred)))

CountVectorized Logistic Regression 의 예측 정확도는 0.608


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


- Count 기반으로 피처 벡터화가 적용된 데이터셋에 대한 로지스틱 회귀 예측 정확도는 약 0.608이다.
- 이번에는 TF-IDF 기반으로 벡터화를 하여 예측 모델을 수행해보자.

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer

# TF-IDF Vectorization 적용하여 학습 데이터셋과 테스트 데이터 셋 변환. 
tfidf_vect = TfidfVectorizer()
tfidf_vect.fit(X_train)
X_train_tfidf_vect = tfidf_vect.transform(X_train)
X_test_tfidf_vect = tfidf_vect.transform(X_test)

In [11]:
# LogisticRegression을 이용하여 학습/예측/평가 수행. 
lr_clf = LogisticRegression()
lr_clf.fit(X_train_tfidf_vect , y_train)
pred = lr_clf.predict(X_test_tfidf_vect)
print('TF-IDF Logistic Regression 의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test ,pred)))

TF-IDF Logistic Regression 의 예측 정확도는 0.674


- TF-IDF가 단순 카운트 기반보다 훨씬 높은 예측 정확도를 제공한다. 일반적으로 문서 내에 텍스트가 많고 문서 개수가 많은 텍스트 분석에서는 TF-IDF 벡터화가 Count 벡터화보다 좋은 예측 결과를 도출한다.
- 텍스트 분석에서 머신러닝 모델의 성능을 향상시키는 중요한 2가지 방법은 최적의 ML 알고리즘을 선택하는 것과 최상의 피처 전처리를 수행하는 것이다.
- 텍스트 정규화나 Count/TF-IDF 기반 피처 벡터화를 어떻게 효과적으로 적용했는지가 텍스트 기반의 머신러닝 성능에 큰 영향을 미칠 수 있다.
- 이번에는 좀 더 다양한 파라미터를 적용하여 TF-IDF 벡터화를 해보자.
  + 스톱 워드를 `'None'`에서 `'english'`로 변경한다.
  + `ngram_range`를 기존 (1,1)에서 (1,2)로 변경한다.
  + `max_df`는 300으로 변경한다.

In [13]:
# stop words 필터링을 추가하고 ngram을 기본(1,1)에서 (1,2)로 변경하여 Feature Vectorization 적용
tfidf_vect = TfidfVectorizer(stop_words='english', ngram_range=(1,2), max_df=300 )
tfidf_vect.fit(X_train)
X_train_tfidf_vect = tfidf_vect.transform(X_train)
X_test_tfidf_vect = tfidf_vect.transform(X_test)

lr_clf = LogisticRegression()
lr_clf.fit(X_train_tfidf_vect , y_train)
pred = lr_clf.predict(X_test_tfidf_vect)
print('TF-IDF Vectorized Logistic Regression 의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test ,pred)))

TF-IDF Vectorized Logistic Regression 의 예측 정확도는 0.692


- 이번에는 GridSearchCV를 이용해 로지스틱 회귀의 하이퍼 파라미터 최적화를 수행해보자. 로지스틱 회귀의 C 파라미터만 변경해서 최적의 C값을 찾은 뒤 이 값으로 학습된 모델에서 테스트 데이터로 예측해 모델 성능을 평가한다.

In [14]:
from sklearn.model_selection import GridSearchCV

# 최적 C 값 도출 튜닝 수행. CV는 3 Fold셋으로 설정. 
params = { 'C':[0.01, 0.1, 1, 5, 10]}
grid_cv_lr = GridSearchCV(lr_clf ,param_grid=params , cv=3 , scoring='accuracy' , verbose=1 )
grid_cv_lr.fit(X_train_tfidf_vect , y_train)
print('Logistic Regression best C parameter :',grid_cv_lr.best_params_ )

# 최적 C 값으로 학습된 grid_cv로 예측 수행하고 정확도 평가. 
pred = grid_cv_lr.predict(X_test_tfidf_vect)
print('TF-IDF Vectorized Logistic Regression 의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test ,pred)))

Fitting 3 folds for each of 5 candidates, totalling 15 fits


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


Logistic Regression best C parameter : {'C': 10}
TF-IDF Vectorized Logistic Regression 의 예측 정확도는 0.701


- 로지스틱 회귀의 C가 10일 때 GridSearchCV의 교차 검증 테스트셋에서 가장 좋은 예측 성능을 보였으며, 이를 테스트 데이터셋에 적용하면, 약 0.703으로 이전보다 약간 향상된 성능을 보여준다.

### | **사이킷런 파이프라인(Pipeline) 사용 및 GridSearchCV와의 결합**
- 사이킷런의 `Pipeline` 클래스를 이용하면 피처 벡터화와 ML 알고리즘 학습/예측을 위한 코드 작성을 한 번에 진행할 수 있다.
- 일반적으로 머신러닝에서 Pipeline이란 데이터의 가공, 변환 등의 전처리와 알고리즘 적용을 마치 **'수도관(Pipe)'에서 물이 흐르듯' 한꺼번에 스트림 기반으로 처리**한다는 의미이다.
- `Pipiline` 사용의 장점
  - 이렇게 `Pipeline`을 이용하면 데이터의 전처리와 머신러닝 학습 과정을 통일된 API 기반에서 처리할 수 있어 더 직관적인 ML 모델 코드를 생성할 수 있다.
  - 또한 대용량 데이터의 피처 벡터화 결과를 별도의 데이터로 저장하지 않고, 스트림 기반에서 바로 머신러닝 알고리즘의 데이터로 입력할 수 있어 수행 시간을 절약할 수 있다.
- 일반적으로 사이킷런의 `Pipeline`은 텍스트 기반의 피처 벡터화뿐만 아니라 모든 데이터 전처리 작업과 Estimator를 결합할 수 있다.
  + 예를 들어 스케일링 또는 벡터 정규화, PCA 등의 변환 작업과 분류, 회귀 등의 Estimator를 한 번에 결합
- 다음은 위에서 텍스트 분류 예제 코드를 `Pipeline`을 이용해 다시 작성한 코드이다.
```python
pipeline = Pipeline([('tfidf_vect', TfidVectorizer(stop_words='english')),
                        ('lr_clf', LogisticRegression(random_state=156))])
```
  + `TfidVectorizer` 객체를 `tfidf_vect`라는 객체 변수 명으로, `LogisticRegression` 객체를 `lr_clf`라는 객체 변수명으로 생성한다.
  + 그리고 두 객체를 파이프라인으로 연결하는 `Pipeline` 객체 `pipeline`을 생성한다.
- 아래의 코드는 피처 벡터화와 머신러닝 모델의 학습과 예측이 Pipeline의 `fit()`과 `transform()`으로 통일되어 수행한다.

In [15]:
from sklearn.pipeline import Pipeline

# TfidfVectorizer 객체를 tfidf_vect 객체명으로, LogisticRegression객체를 lr_clf 객체명으로 생성하는 Pipeline생성
pipeline = Pipeline([
    ('tfidf_vect', TfidfVectorizer(stop_words='english', ngram_range=(1,2), max_df=300)),
    ('lr_clf', LogisticRegression(C=10))
])

# 별도의 TfidfVectorizer객체의 fit_transform( )과 LogisticRegression의 fit(), predict( )가 필요 없음 
# pipeline의 fit() 과 predict() 만으로 한꺼번에 Feature Vectorization과 ML 학습/예측이 가능
pipeline.fit(X_train, y_train)
pred = pipeline.predict(X_test)

print('Pipeline을 통한 Logistic Regression 의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test ,pred)))

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


Pipeline을 통한 Logistic Regression 의 예측 정확도는 0.701


- 사이킷런은 `GridSearchCV` 클래스의 생성 파라미터로 `Pipeline`을 입력하여, `Pipeline` 기반에서도 하이퍼 파라미터 튜닝을 그리드서치 방식으로 진행할 수 있게 지원한다. 이렇게 하면 피처 벡터화를 위한 파라미터와 ML 알고리즘의 하이퍼 파라미터를 한 번에 `GridSearchCV`를 이용하여 최적화할 수 있다.
- 다음 예제에서는 `GridSearchCV`에 `Pipeline`을 입력하여 `TfidfVectorizer`의 파라미터와 `LogisticRegression`의 하이퍼파라미터를 함께 최적화한다.
  + `GridSearchCV`에 Estimator가 아닌 Pipeline을 입력할 경우, `param_grid`의 입력 값 설정이 하이퍼 파라미터명이 객체 변수명과 결합되어 제공된다는 점에서 기존과 조금 다르다. 
- `GridSearchCV`에 `Pipeline`을 적용할 때 유의할 점은 모든 파라미터를 최적화하면 시간이 많이 걸린다는 점이다.

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

pipeline = Pipeline([
    ('tfidf_vect', TfidfVectorizer(stop_words='english')),
    ('lr_clf', LogisticRegression())
])

# Pipeline에 기술된 각각의 객체 변수에 언더바(_)2개를 연달아 붙여 GridSearchCV에 사용될 
# 파라미터/하이퍼 파라미터 이름과 값을 설정. . 
params = { 'tfidf_vect__ngram_range': [(1,1), (1,2), (1,3)],
           'tfidf_vect__max_df': [100, 300, 700],
           'lr_clf__C': [1,5,10]
}

# GridSearchCV의 생성자에 Estimator가 아닌 Pipeline 객체 입력
grid_cv_pipe = GridSearchCV(pipeline, param_grid=params, cv=3 , scoring='accuracy',verbose=1)
grid_cv_pipe.fit(X_train , y_train)
print(grid_cv_pipe.best_params_ , grid_cv_pipe.best_score_)

pred = grid_cv_pipe.predict(X_test)
print('Pipeline을 통한 Logistic Regression 의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test ,pred)))

Fitting 3 folds for each of 27 candidates, totalling 81 fits


- 로지스틱 회귀 외에 서포트 벡터머신과 나이브 베이즈 알고리즘도 희소 행렬 기반의 텍스트 분류에서 자주 사용되는 머신러닝 알고리즘이다.